In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
findspark.find()
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier

from sklearn.metrics import confusion_matrix, precision_score, recall_score

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls "/content/gdrive/My Drive/Bigdata/data"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
mushrooms.csv  plants.data   products.csv
orders.csv     plants.names  stateabbr.txt


In [ ]:
DATA_PATH = "/content/gdrive/My Drive/Bigdata/data/mushrooms.csv" 

In [ ]:
data = spark.read.load(DATA_PATH, format="csv", header=True, delimiter=",")
data.show()

+-----+---------+-----------+---------+-------+----+---------------+------------+---------+----------+-----------+----------+------------------------+------------------------+----------------------+----------------------+---------+----------+-----------+---------+-----------------+----------+-------+
|class|cap-shape|cap-surface|cap-color|bruises|odor|gill-attachment|gill-spacing|gill-size|gill-color|stalk-shape|stalk-root|stalk-surface-above-ring|stalk-surface-below-ring|stalk-color-above-ring|stalk-color-below-ring|veil-type|veil-color|ring-number|ring-type|spore-print-color|population|habitat|
+-----+---------+-----------+---------+-------+----+---------------+------------+---------+----------+-----------+----------+------------------------+------------------------+----------------------+----------------------+---------+----------+-----------+---------+-----------------+----------+-------+
|    p|        x|          s|        n|      t|   p|              f|           c|        n|   

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer
columns = ['cap-shape', 'cap-surface', 'cap-color', 'bruises',\
                   'odor', 'gill-attachment', 'gill-spacing', 'gill-size',\
                   'gill-color', 'stalk-shape', 'stalk-root',\
                   'stalk-surface-above-ring', 'stalk-surface-below-ring',\
                   'stalk-color-above-ring', 'stalk-color-below-ring',\
                   'veil-type', 'veil-color', 'ring-number', 'ring-type',\
                   'spore-print-color', 'population', 'habitat']

indexers = [StringIndexer(inputCol=column, outputCol=column+"_numeric").fit(data) for column in columns]

pipeline = Pipeline(stages=indexers)
df_indexed = pipeline.fit(data).transform(data)
df_indexed.show()

+-----+---------+-----------+---------+-------+----+---------------+------------+---------+----------+-----------+----------+------------------------+------------------------+----------------------+----------------------+---------+----------+-----------+---------+-----------------+----------+-------+-----------------+-------------------+-----------------+---------------+------------+-----------------------+--------------------+-----------------+------------------+-------------------+------------------+--------------------------------+--------------------------------+------------------------------+------------------------------+-----------------+------------------+-------------------+-----------------+-------------------------+------------------+---------------+
|class|cap-shape|cap-surface|cap-color|bruises|odor|gill-attachment|gill-spacing|gill-size|gill-color|stalk-shape|stalk-root|stalk-surface-above-ring|stalk-surface-below-ring|stalk-color-above-ring|stalk-color-below-ring|veil-ty

In [ ]:
feature_columns = [col + '_numeric' for col in columns]
feature_columns

['cap-shape_numeric',
 'cap-surface_numeric',
 'cap-color_numeric',
 'bruises_numeric',
 'odor_numeric',
 'gill-attachment_numeric',
 'gill-spacing_numeric',
 'gill-size_numeric',
 'gill-color_numeric',
 'stalk-shape_numeric',
 'stalk-root_numeric',
 'stalk-surface-above-ring_numeric',
 'stalk-surface-below-ring_numeric',
 'stalk-color-above-ring_numeric',
 'stalk-color-below-ring_numeric',
 'veil-type_numeric',
 'veil-color_numeric',
 'ring-number_numeric',
 'ring-type_numeric',
 'spore-print-color_numeric',
 'population_numeric',
 'habitat_numeric']

In [ ]:
(train, test) = df_indexed.randomSplit([0.8, 0.2])

In [ ]:
# Tạo assembler gom các giá trị ở các cột dùng làm features thành
# các vector ở cột "features"
assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')

# Tạo indexer để tạo index cho các vector ở cột "features"
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=50)

# Tạo indexer để map các giá trị cột "Label" sang chỉ số nhãn (label indices)
labelIndexer = StringIndexer(inputCol='class', outputCol='indexedLabel')

# Tạo một bộ phân loại decision tree
dt = DecisionTreeClassifier(featuresCol="indexedFeatures", labelCol="indexedLabel")

# Xây dựng pipeline là chuỗi các assembler, indexer và classifier
pipeline = Pipeline(stages=[assembler, featureIndexer, labelIndexer, dt])

# Training với pipeline đã tạo
model = pipeline.fit(train)

In [ ]:
# Sử dụng model đã train để dự đoán trên tập test
predictions = model.transform(test)
# In ra cột features, giá trị dự đoán và nhãn
predictions.select("features", "prediction", "indexedLabel").show(5)

+--------------------+----------+------------+
|            features|prediction|indexedLabel|
+--------------------+----------+------------+
|(22,[0,1,2,6,8,9,...|       0.0|         0.0|
|(22,[0,1,2,6,8,9,...|       0.0|         0.0|
|(22,[0,1,2,6,8,9,...|       0.0|         0.0|
|(22,[0,1,2,6,8,9,...|       0.0|         0.0|
|(22,[0,1,2,6,8,9,...|       0.0|         0.0|
+--------------------+----------+------------+
only showing top 5 rows



In [ ]:
# Tạo một Evaluator và tính độ chính xác dự đoán của model
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

# In thông tin summary của model
treeModel = model.stages[-1]
print(treeModel)
# In các kết quả đánh giá model
print("Decision Tree - Test Accuracy = %g" % (accuracy))
print("Decision Tree - Test Error = %g" % (1.0 - accuracy))

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_31490bd4e1f8, depth=4, numNodes=9, numClasses=2, numFeatures=22
Decision Tree - Test Accuracy = 0.996224
Decision Tree - Test Error = 0.00377596


In [ ]:
# Tạo y_true, y_pred kiểu list để đánh giá model sử dụng các hàm của sklearn
y_true = predictions.select("indexedLabel").rdd.flatMap(lambda x: x).collect()
y_pred = predictions.select("prediction").rdd.flatMap(lambda x: x).collect()

# Tính toán confusion matrix, precision và recall của model
confusionmatrix = confusion_matrix(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='micro')
recall = recall_score(y_true, y_pred, average='micro')

print("The Confusion Matrix for Decision Tree Model is :\n" + str(confusionmatrix))
print("The precision score for Decision Tree Model is: " + str(precision))
print("The recall score for Decision Tree Model is: " + str(recall))

The Confusion Matrix for Decision Tree Model is :
[[837   0]
 [  6 746]]
The precision score for Decision Tree Model is: 0.9962240402769037
The recall score for Decision Tree Model is: 0.9962240402769037


# Random Forest

In [ ]:
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# StringIndexer để mã hóa cột label thành một cột chỉ số nhãn.
# Theo mặc định, các nhãn được gán theo tần số. Vì vậy, 
# label xuất hiện thường xuyên nhất nhận được chỉ số 0.
labelIndexer = StringIndexer(inputCol="class", outputCol="indexedLabel")

# Tự động phân loại các features thành maxCategories nhóm.
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4)

# Tạo một bộ phân loại random forest
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

# Xây dựng pipeline
pipeline = Pipeline(stages=[assembler, labelIndexer, featureIndexer, rf])

# Train model.
model = pipeline.fit(train)

In [ ]:
# Sử dụng model đã train để dự đoán trên tập test
predictions = model.transform(test)
# Chọn một vài cột để thấy được kết quả
predictions.select("features", "prediction", "indexedLabel").show(5)

+--------------------+----------+------------+
|            features|prediction|indexedLabel|
+--------------------+----------+------------+
|(22,[0,1,2,6,8,9,...|       0.0|         0.0|
|(22,[0,1,2,6,8,9,...|       0.0|         0.0|
|(22,[0,1,2,6,8,9,...|       0.0|         0.0|
|(22,[0,1,2,6,8,9,...|       0.0|         0.0|
|(22,[0,1,2,6,8,9,...|       0.0|         0.0|
+--------------------+----------+------------+
only showing top 5 rows



In [ ]:
# Đánh giá thuật toán 
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")

accuracy = evaluator.evaluate(predictions)
print("Random Forest - Test Accuracy = %g" % (accuracy))
print("Random Forest - Test Error = %g" % (1.0 - accuracy))

Random Forest - Test Accuracy = 0.998764
Random Forest - Test Error = 0.00123609


In [ ]:
# Tạo y_true, y_pred kiểu list để đánh giá model sử dụng các hàm của sklearn
y_true = predictions.select("indexedLabel").rdd.flatMap(lambda x: x).collect()
y_pred = predictions.select("prediction").rdd.flatMap(lambda x: x).collect()

# Tính toán confusion matrix, precision và recall của model
confusionmatrix = confusion_matrix(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='micro')
recall = recall_score(y_true, y_pred, average='micro')

print("The Confusion Matrix for Random Forest Model is :\n" + str(confusionmatrix))
print("The precision score for Random Forest Model is: " + str(precision))
print("The recall score for Random Forest Model is: " + str(recall))

The Confusion Matrix for Random Forest Model is :
[[830   0]
 [  2 786]]
The precision score for Random Forest Model is: 0.9987639060568603
The recall score for Random Forest Model is: 0.9987639060568603
